# pilon correct FLYE assembly

* This was done twice to achieve results from two rounds of error correction.
* current scripts are from the second round of error correction.

## 1. mapping

In [5]:
CTG_PFR_AGRF=/workspace/hraczw/github/hoki_genomics/Flye_All_plusP/assembly.ml1000.fasta
CTG_PFR_AGRF_P1=/workspace/hraczw/github/hoki_genomics/005.pilon_correctFLYE/FlyeAssembly_PFR_AGRF_plusP_pilon-1_corrected.fasta
R1=/input/genomic/fish/Macruronus/novaezelandiae/Illumina/AGRF_CAGRF20062892_J9Y9G/HHP2_J9Y9G_AACGTTCC-GGAGTACT_L001_R1.fastq.gz
R2=/input/genomic/fish/Macruronus/novaezelandiae/Illumina/AGRF_CAGRF20062892_J9Y9G/HHP2_J9Y9G_AACGTTCC-GGAGTACT_L001_R2.fastq.gz

In [2]:
module load bowtie2
module list

Currently Loaded Modulefiles:
  1) powerPlant/core    4) git/2.21.0         7) asub/2.2
  2) texlive/20151117   5) perlbrew/0.76      8) samtools/1.9
  3) pandoc/1.19.2      6) perl/5.28.0        9) bowtie2/2.3.4.3


In [3]:
mkdir 005.pilon_correctFLYE

In [6]:
WORKDIR=005.pilon_correctFLYE

In [20]:
bsub -J bowtie2-build \
    -n 8 \
    -o $WORKDIR/bowtie2-build_PFR_AGRF_plusP_P1.out \
    -e $WORKDIR/bowtie2-build_PFR_AGRF_plusP_P1.err \
    "bowtie2-build \
    --threads 8 $CTG_PFR_AGRF_P1 $WORKDIR/FlyeAssembly_PFR_AGRF_plusP_P1"

Job <35828> is submitted to default queue <lowpriority>.


In [7]:
module list

Currently Loaded Modulefiles:
  1) powerPlant/core    4) git/2.21.0         7) asub/2.2
  2) texlive/20151117   5) perlbrew/0.76      8) bowtie2/2.3.4.3
  3) pandoc/1.19.2      6) perl/5.28.0


In [1]:
module load samtools
module list

Currently Loaded Modulefiles:
  1) powerPlant/core    4) git/2.21.0         7) asub/2.2
  2) texlive/20151117   5) perlbrew/0.76      8) samtools/1.9
  3) pandoc/1.19.2      6) perl/5.28.0


In [22]:
bsub -J mapping \
                -n 36 \
                -o $WORKDIR/bowtie2_PFR_AGRF_plusP_P1.out \
                -e $WORKDIR/bowtie2_PFR_AGRF_plusP_P1.err \
                "bowtie2 \
                -p 36 \
                -x $WORKDIR/FlyeAssembly_PFR_AGRF_plusP_P1 \
                -1 $R1 \
                -2 $R2 | \
                samtools view -bS - > $WORKDIR/FlyeAssembly_PFR_AGRF_plusP_P1.bam"

Job <35829> is submitted to default queue <lowpriority>.


In [7]:
bsub -J sorting \
    -n 10 \
    -o $WORKDIR/sort_PFR_AGRF_plusP_P1.out \
    -e $WORKDIR/sort_PFR_AGRF_plusP_P1.err \
    "samtools sort \
    -@ 10 \
    -o $WORKDIR/FlyeAssembly_PFR_AGRF_plusP_P1.sorted.bam \
    $WORKDIR/FlyeAssembly_PFR_AGRF_plusP_P1.bam"

Job <37583> is submitted to default queue <lowpriority>.


In [8]:
bsub -J indexing \
    -o $WORKDIR/index_PFR_AGRF_plusP_P1.out \
    -e $WORKDIR/index_PFR_AGRF_plusP_P1.err \
    "samtools index $WORKDIR/FlyeAssembly_PFR_AGRF_plusP_P1.sorted.bam"

Job <37658> is submitted to default queue <lowpriority>.


## 2. pilon correction

In [9]:
bsub -J pilon \
    -m wkoppb50 \
    -n 60 \
    -o $WORKDIR/pilon-1_PFR_AGRF_plusP_P1.out \
    -e $WORKDIR/pilon-1_PFR_AGRF_plusP_P1.err \
    "taskset -a -c 0-60 java -Xmx1500G -jar /software/bioinformatics/pilon-1.23/pilon-1.23.jar \
     --genome $CTG_PFR_AGRF_P1 \
     --frags $WORKDIR/FlyeAssembly_PFR_AGRF_plusP_P1.sorted.bam \
     --output FlyeAssembly_PFR_AGRF_plusP_P1_pilon-2_corrected \
     --outdir $WORKDIR/ \
     --changes \
     --diploid \
     --fix all \
     --threads 60 \
     --flank 5"

Job <37662> is submitted to default queue <lowpriority>.
